In [1]:
import pandas as pd
import numpy as np
import joblib
import pickle
import mlflow.sklearn

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [18]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv( r"C:\Users\axa\THE BRIDGE_23\SEMANA 24. CORE. PROYECTO ML\81. PROYECTO ML\src\data\raw\online_shoppers_intention.csv", sep=",")

## Transformación de Variables Booleanas a Númericas

* La mayoría de los algoritmos de ML: (regresión, clasificación, clustering y otros), trabajan con variables numéricas. Con variables booleanas en su forma original (True/False o 1/0), los algoritmos pueden funcionar sin problemas.
* Simplifica la preparación de datos y facilita la modelización. Los valores numéricos son más fáciles de analizar y de utilizar en cálculos matemáticos.
* Favorecen la compatibilidad con Bibliotecas y Herramientas: Las bibliotecas de ML (Scikit-Learn en Python), y otras herramientas estadísticas están diseñadas para trabajar con datos numéricos.
* En algunos casos, transformar variables booleanas puede mejorar el rendimiento del modelo. Algunos algoritmos pueden beneficiarse de la representación numérica de las variables booleanas.

In [8]:
le = LabelEncoder()

In [9]:
df["Weekend"] = le.fit_transform(df["Weekend"])

In [10]:
df["Revenue"] = le.fit_transform(df["Revenue"])

## Transformación de variables "Objeto" a Variables Dummies

* Al usar variables dummies, los coeficientes en un modelo (como regresión lineal) se vuelven más interpretables. 
* Cada coeficiente representa el efecto de esa categoría específica en comparación con la categoría de referencia (la original). La eliminación de una columna facilita la interpretación.
* La multicolinealidad se refiere a la correlación alta o perfecta entre dos o más variables predictoras en un modelo estadístico. 
* En el caso de variables dummies, si se tienen múltiples columnas que representan una característica categórica con dos categorías, la presencia de una categoría se puede predecir a partir de la otra. Esto crea multicolinealidad y puede causar problemas en la interpretación y estabilidad del modelo.
* Si no se elimina una de las columnas de las variables ficticias, existe la "trampa de las variables ficticias" (dummy variable trap). Esto significa que las variables predictoras son linealmente dependientes y el modelo no puede identificar coeficientes únicos para cada una. Eliminar una columna resuelve este problema.
* Compatibilidad con Modelos y Bibliotecas

In [11]:
# Transformar la columna "Month" en variables Dummies (sólo hay 10 meses, y eliminar la primera columna de las 10 para evitar Multicolinealidad)
df_dummies_Month = pd.get_dummies(df, columns=['Month'], prefix='Month', drop_first=True)

In [12]:
# Transformar la columna "VisitorType" en variables Dummies y eliminar la primera columna para evitar Multicolinealidad
df_transformado = pd.get_dummies(df_dummies_Month, columns=['VisitorType'], prefix='VisitorType', drop_first=True)

## Estandarizar / Escalar / Normalizar

* Igualdad de peso: Cuando las variables numéricas tienen diferentes escalas, algunos algoritmos de ML pueden tratar a las variables con valores más grandes como más importantes. Estandarizar las variables garantiza que todas tengan un rango similar, lo que permite que el modelo trate todas las características por igual.
* La estandarización ayuda a que los algoritmos converjan más rápido y alcancen resultados más precisos.
* La estandarización facilita la interpretación de coeficientes en modelos lineales. Los coeficientes estandarizados se pueden comparar directamente para evaluar la importancia relativa de las características.
* Algunos algoritmos numéricos pueden ser propensos a errores numéricos si los datos tienen magnitudes muy diferentes. La estandarización reduce el riesgo de tales errores.
* La estandarización de las variables numéricas es una buena práctica que puede mejorar el rendimiento de los modelos y garantizar resultados más consistentes y significativos en el análisis de datos y ML.
* Al estandarizar con StandardScaler, los datos se centran en cero, lo que significa que la media de las variables escaladas será cero. Esto es útil para algoritmos sensibles a la media, como la regresión lineal. El centrado en cero también facilita la interpretación de los coeficientes en modelos lineales.
* Después de la estandarización, todas las variables tienen una desviación estándar de 1. Esto asegura que todas las variables tengan la misma escala.
* El StandardScaler es adecuado para una amplia variedad de datos, incluidos los datos que se asemejan a una distribución normal, ya que no asume una distribución específica para los datos.
* El StandardScaler es moderadamente robusto ante valores atípicos, (no se verá significativamente afectado por observaciones atípicas). Si los datos contienen valores extremos, esta técnica es efectiva.

In [13]:
caracteristicas_a_escalar = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration',
                            'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay',
                            'OperatingSystems','Browser', 'Region', 'TrafficType', "Weekend"]

caracteristicas_numericas = df_transformado[caracteristicas_a_escalar]

sc = StandardScaler()

escaladas = sc.fit_transform(caracteristicas_numericas)

df_transformado[caracteristicas_a_escalar] = escaladas

In [14]:
df_transformado

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_Other,VisitorType_Returning_Visitor
0,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,...,True,False,False,False,False,False,False,False,False,True
1,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.317178,-0.308821,...,True,False,False,False,False,False,False,False,False,True
2,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,...,True,False,False,False,False,False,False,False,False,True
3,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.622954,0.573535,1.994610,-0.317178,-0.308821,...,True,False,False,False,False,False,False,False,False,True
4,-0.696993,-0.457191,-0.396478,-0.244931,-0.488636,-0.296430,-0.045196,0.142551,-0.317178,-0.308821,...,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.206173,0.363075,-0.396478,-0.244931,0.478227,0.307822,-0.310366,-0.288966,0.342125,-0.308821,...,False,False,False,False,False,False,False,False,False,True
12326,-0.696993,-0.457191,-0.396478,-0.244931,-0.601062,-0.380957,-0.457683,-0.447364,-0.317178,-0.308821,...,False,False,False,False,False,True,False,False,False,True
12327,-0.696993,-0.457191,-0.396478,-0.244931,-0.578577,-0.528063,1.261014,0.897093,-0.317178,-0.308821,...,False,False,False,False,False,True,False,False,False,True
12328,0.507228,-0.032916,-0.396478,-0.244931,-0.376210,-0.443536,-0.457683,-0.453140,-0.317178,-0.308821,...,False,False,False,False,False,True,False,False,False,True


In [15]:
ruta_guardado = r"C:\Users\axa\THE BRIDGE_23\SEMANA 24. CORE. PROYECTO ML\81. PROYECTO ML\src\data\processed\datos_procesados.csv"

df_transformado.to_csv(ruta_guardado, index=False)

print(f"El archivo CSV se ha guardado en: {ruta_guardado}")

El archivo CSV se ha guardado en: C:\Users\axa\THE BRIDGE_23\SEMANA 24. CORE. PROYECTO ML\81. PROYECTO ML\src\data\processed\datos_procesados.csv


## my_model: BalancedRandomForestClassifier

In [16]:
X = df_transformado.drop('Revenue', axis=1)
y = df_transformado['Revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, shuffle=True)

In [ ]:
# Guardar X_train, X_test, y_train, y_test

In [21]:
my_model = BalancedRandomForestClassifier(n_estimators=100, max_depth=5, class_weight= "balanced", random_state=1)
my_model.fit(X_train, y_train)

BalancedRandomForestClassifier(class_weight='balanced', max_depth=5,
                               random_state=1)

In [22]:
joblib.dump(my_model, 'my_model.pkl')

['my_model.pkl']

In [23]:
y_pred = my_model.predict(X_test)

In [24]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, my_model.predict_proba(X_test)[:, 1])
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy my_model:", accuracy)
print("Precision my_model:", precision)
print("Recall my_model:", recall)
print("F1-score my_model:", f1)
print("AUC-ROC my_model:", roc_auc)
print("Matriz de Confusión my_model:")
print(conf_matrix)

Accuracy my_model: 0.5355501486888348
Precision my_model: 0.24113475177304963
Recall my_model: 0.9890909090909091
F1-score my_model: 0.3877405559515324
AUC-ROC my_model: 0.9158913363549757
Matriz de Confusión my_model:
[[1437 1712]
 [   6  544]]


## Trazabilidad  (Incluirla en la fase 4)